# V.1 Exploring the green reds

## a) Write a function that will plot a scatterplot matrix of your red wine data.

In [55]:
def plot_scatter_matrix(winedata, good_threshold, bad_threshold, save_plot=False):
    import matplotlib.pyplot as plt
    import matplotlib.patches as patches
    c = winedata.columns
    cmap = plt.get_cmap("Set2")
    fig, axes = plt.subplots(len(c) - 1, len(c) - 1)
    fig.subplots_adjust(wspace=0, hspace=0)
    for i in range(len(c) - 1):
        for j in range(len(c) - 1):
            ax = axes[i, j]
            p = patches.Rectangle((0, 0), 1, 1, fill=True, transform=ax.transAxes,
                clip_on=False, facecolor='w', zorder=0.1, edgecolor='#000000')
            ax.add_patch(p)
            if (i != j):
                y1 = winedata[winedata['quality'] < bad_threshold][c[i]]
                x1 = winedata[winedata['quality'] < bad_threshold][c[j]]
                y2 = winedata[winedata['quality'] > good_threshold][c[i]]
                x2 = winedata[winedata['quality'] > good_threshold][c[j]]
                min = x1.min()
                if (min > x2.min()):
                    min = x2.min()
                max = x1.max()
                if (max < x2.max()):
                    max = x2.max()
                ax.set_xlim(left=min*0.95, right=max*1.05)
                min = y1.min()
                if (min > y2.min()):
                    min = y2.min()
                max = y1.max()
                if (max < y2.max()):
                    max = y2.max()
                ax.set_ylim(bottom=min*0.95, top=max*1.05)
                ax.scatter(x1, y1, marker='.', s=20, linewidths=0, c='#EC008C')
                ax.scatter(x2, y2, marker='.', s=20, linewidths=0, c='#009E49')
            else:
                ax.text(0.5, 0.5, c[i], horizontalalignment='center', verticalalignment='center',
                    transform=ax.transAxes, size='small')
            ax.set_axis_off()
    plt.show()


## b) Analyze different chemical factors in red wines 

In [ ]:
import pandas as pd
df = pd.read_csv('winequality-red.csv', sep=";")
plot_scatter_matrix(df, 7, 4)

pH and alcohol is most useful to split wines by quality, because they perfectly visualy divided on two different zones.

# V.2 Learning to perceptron

## a) Implement a perceptron